In [1]:
%load_ext autoreload
%autoreload

In [18]:
from load_data import Balance, Instruments, AdvisedPortfolios, PriceDB, Singleton
from utils import get_current_port, get_advised_port, get_recommendation

from asset import Asset
from price import Price
from portfolio import Portfolio

import pandas as pd

In [3]:
userid='A01'
risk_profile=2

#### `userid`로 사용자 최근잔고 잔고 불러오기
- `balance`
- `advised_pf`

In [4]:
balance = get_current_port(userid=userid)
new_port = get_advised_port(risk_profile=risk_profile)

In [5]:
old_assets = balance.loc[(balance.tracking_code != 'CASH') & (balance.tracking_code != 'DEPOSIT'), :]
old_cash = balance.loc[(balance.tracking_code=='CASH') | (balance.tracking_code=='DEPOSIT'), :]
old_tickers = old_assets.itemcode.tolist()
old_quantities = old_assets.quantity.tolist()
old_prices = old_assets.price.tolist()
cash_amounts = old_cash.value.tolist()
cash_currency = ['KRW']*len(cash_amounts)

In [34]:
old_new = pd.merge(balance.loc[:, ['exposure', 'itemcode', 'wt']], new_port.loc[:, ['exposure', 'itemcode', 'wt']],
                left_on=['exposure'], right_on=['exposure'], how='outer', suffixes=['_old', '_new'])

In [37]:
 old_new.loc[old_new.itemcode_old.isna(), 'itemcode_old'] = old_new.loc[old_new.itemcode_old.isna(), 'itemcode_new']

In [38]:
old_new

,exposure,itemcode_old,wt_old,itemcode_new,wt_new
0,CASH,C000001,0.027733,NaN,NaN
1,DEPOSIT,D000001,0.501074,NaN,NaN
2,EQ_M_KR_LARGE,A069500,0.060862,A102110,0.030000
3,EQ_M_US_LARGE,A360200,0.029569,NaN,NaN
4,MM,A196230,0.153792,A130730,0.250000
5,MM,A196230,0.153792,A153130,0.217851
6,MM,A196230,0.153792,A214980,0.100000
7,FI_D_LONG,A302190,0.073483,NaN,NaN
8,AL_FX_USD,A329750,0.109372,NaN,NaN
9,EQ_S_NASDAQ,A367380,0.044114,NaN,NaN


In [30]:
old_new

,exposure,itemcode_old,wt_old,itemcode_new,wt_new
0,CASH,C000001,0.027733,NaN,NaN
1,DEPOSIT,D000001,0.501074,NaN,NaN
2,EQ_M_KR_LARGE,A069500,0.060862,A102110,0.030000
3,EQ_M_US_LARGE,A360200,0.029569,NaN,NaN
4,MM,A196230,0.153792,A130730,0.250000
5,MM,A196230,0.153792,A153130,0.217851
6,MM,A196230,0.153792,A214980,0.100000
7,FI_D_LONG,A302190,0.073483,NaN,NaN
8,AL_FX_USD,A329750,0.109372,NaN,NaN
9,EQ_S_NASDAQ,A367380,0.044114,NaN,NaN


In [6]:
p = Portfolio()
p.easy_add_assets(tickers=old_tickers, quantities=old_quantities, prices=old_prices)
p.easy_add_cash(amounts=cash_amounts, currencies=cash_currency)
p.selling_allowed = True

In [7]:
p.assets

{'A069500': <asset.Asset at 0x1e0cead8b20>,
 'A360200': <asset.Asset at 0x1e0cead8be0>,
 'A196230': <asset.Asset at 0x1e0cead8cd0>,
 'A302190': <asset.Asset at 0x1e0cead8d90>,
 'A329750': <asset.Asset at 0x1e0cead8e80>,
 'A367380': <asset.Asset at 0x1e0cead8f40>}

In [9]:
new_tickers = new_port.itemcode.tolist()
new_quantities = (new_port.wt*100).tolist()     # 단위가 %이므로 100을 곱한다.

In [10]:
new_tickers

['A122260', 'A130730', 'A153130', 'A114260', 'A214980', 'A102110']

In [11]:
target_asset_alloc = dict(zip(new_tickers, new_quantities))    

In [12]:
p.rebalance(target_asset_alloc, verbose=True)

Exception: 'target_allocation not compatible with the assets of the portfolio.

In [16]:
target_asset_alloc['A069500']

KeyError: 'A069500'

In [13]:
p.assets

{'A069500': <asset.Asset at 0x1e0cead8b20>,
 'A360200': <asset.Asset at 0x1e0cead8be0>,
 'A196230': <asset.Asset at 0x1e0cead8cd0>,
 'A302190': <asset.Asset at 0x1e0cead8d90>,
 'A329750': <asset.Asset at 0x1e0cead8e80>,
 'A367380': <asset.Asset at 0x1e0cead8f40>}

In [63]:
new_port.wt.sum()

0.9999999998925693

In [68]:
import numpy as np

In [70]:
# order target_allocation dict in the same order as assets dict and upper key
target_allocation_reordered = {}
try:
    for key in assets:
        target_allocation_reordered[key] = target_allocation[key]
except:
    raise Exception(
        "'target_allocation not compatible with the assets of the portfolio."
    )

target_allocation_np = np.fromiter(
    target_allocation_reordered.values(), dtype=float)

Exception: 'target_allocation not compatible with the assets of the portfolio.

In [73]:
p.assets

{}